In [ ]:
from sentinelsat.sentinel import SentinelAPI
import cdsapi
import cfgrib
import xarray as xr
import pandas as pd
import numpy as np
from itertools import product
import scipy.interpolate
from copy import copy
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import cartopy.crs as ccrs
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
import cartopy.feature as cfeature

In [ ]:
def nearest_neighbour(array, value):

    # Get indexes of the closest latitude and longitude
    index = np.abs([x - value for x in array]).argmin(0)
    
    return index

In [ ]:
def CAMS_download(start_date, end_date, component, component_nom):

    # Access server 
    c = cdsapi.Client()

    # Retrieve GRIB file
    c.retrieve(
        'cams-global-atmospheric-composition-forecasts',
        {
            'date': start_date + '/' + end_date,
            'type': 'forecast',
            'format': 'grib',
            'variable': component,
            'model_level': [
                '1', '2', '3',
                '4', '5', '6',
                '7', '8', '9',
                '10', '11', '12',
                '13', '14', '15',
                '16', '17', '18',
                '19', '20', '21',
                '22', '23', '24',
                '25', '26', '27',
                '28', '29', '30',
                '31', '32', '33',
                '34', '35', '36',
                '37', '38', '39',
                '40', '41', '42',
                '43', '44', '45',
                '46', '47', '48',
                '49', '50', '51',
                '52', '53', '54',
                '55', '56', '57',
                '58', '59', '60',
                '61', '62', '63',
                '64', '65', '66',
                '67', '68', '69',
                '70', '71', '72',
                '73', '74', '75',
                '76', '77', '78',
                '79', '80', '81',
                '82', '83', '84',
                '85', '86', '87',
                '88', '89', '90',
                '91', '92', '93',
                '94', '95', '96',
                '97', '98', '99',
                '100', '101', '102',
                '103', '104', '105',
                '106', '107', '108',
                '109', '110', '111',
                '112', '113', '114',
                '115', '116', '117',
                '118', '119', '120',
                '121', '122', '123',
                '124', '125', '126',
                '127', '128', '129',
                '130', '131', '132',
                '133', '134', '135',
                '136', '137',
            ],
            'time': '00:00',
            'leadtime_hour': [
                '0', '12', '18', '6', 
            ],
        },
        'data/ads/' + component_nom + '/' + component_nom + '-' + start_date + '-' + end_date + '.grib')

In [ ]:
def CAMS_read(start_date, end_date, component, component_nom):
    
    # Read as xarray dataset object
    CAMS_ds = xr.open_dataset('data/ads/' + component_nom + '/' + component_nom + '-levels-' + 
                            start_date + '-' + end_date + '.grib')

    # Change longitude coordinates
    CAMS_ds = CAMS_ds.assign_coords(longitude = (((CAMS_ds.longitude + 180) % 360) - 180)).sortby('longitude')
    CAMS_ds = CAMS_ds.sortby('latitude')

    # Change name to component
    CAMS_ds = CAMS_ds.rename({'no2': 'component'})

    return CAMS_ds

In [ ]:
def CAMS_interpolation(CAMS_ds, TROPOMI_ds, lon_min, lon_max, lat_min, lat_max, component_nom):

    # INTERPOLATION

    # Grid data from CAMS
    x = CAMS_ds.longitude.values
    y = CAMS_ds.latitude.values
    x_old, y_old = np.meshgrid(x, y)

    # Grid data in 100x100
    xi = np.linspace(lon_min, lon_max, 100)
    yi = np.linspace(lat_min, lat_max, 100)
    x_new, y_new = np.meshgrid(xi, yi)

    da_hybrid = []
    da_step = []

    for step in range(CAMS_ds.step.size):

        for hybrid in range(CAMS_ds.hybrid.size):
            
            z = CAMS_ds.isel(hybrid = hybrid, step = step).component.values

            zi = scipy.interpolate.griddata((x_old.flatten(), y_old.flatten()), 
                                            z.flatten(), (xi[None,:], yi[:,None]), 
                                            method = 'linear')

            # Create data array for each layer
            da = xr.DataArray(data = xr.Variable(('lon', 'lat'), zi),
                            dims = ['lon', 'lat'],
                            coords = {'longitude': xr.Variable('lon', xi),
                                        'latitude': xr.Variable('lat', yi)
                                    }
            )

            # Append arrays for each layer
            da_hybrid.append(da)

        # Concatenate data arrays for all layers
        da_hybrid_concat = xr.concat(da_hybrid, pd.Index(range(CAMS_ds.hybrid.size), 
                                                        name = 'hybrid'))

        da_step.append(da_hybrid_concat)
        da_hybrid = []

    CAMS_ds_new = xr.concat(da_step, pd.Index(CAMS_ds.valid_time.values, 
                                            name = 'valid_time'))

    # VISUALIZATION

    z_1L = CAMS_ds.isel(hybrid = 136, step = 2).component.values
    zi_1L = CAMS_ds_new.isel(hybrid = 136, valid_time = 2).values

    fig, ax = plt.subplots(nrows = 1, ncols = 2, figsize = (20, 10))

    # Show old CAMS grid
    im1 = ax[0].scatter(x_old, y_old, c = z_1L, cmap = 'coolwarm', vmin = np.nanmin(z_1L), vmax = np.nanmax(z_1L))

    # Show contour plot of new CAMS data
    im2 = ax[1].contourf(x_new, y_new, zi_1L, cmap = 'coolwarm', vmin = np.nanmin(zi_1L), vmax = np.nanmax(zi_1L))

    # Show new CAMS grid
    ax[1].scatter(x_new, y_new, marker = 'o', c = 'grey', s = 3)

    # Show TROPOMI grid
    TROPOMI_lat = TROPOMI_ds['latitude'].values
    TROPOMI_lon = TROPOMI_ds['longitude'].values
    ax[1].scatter(TROPOMI_lon, TROPOMI_lat, marker = 'x', c = 'black', s = 30)

    # Add colorbars
    cbr1 = fig.colorbar(im1, ax = ax[0])
    cbr2 = fig.colorbar(im2, ax = ax[1])
    cbr1.set_label(f'{component_nom} (mol/m²)', fontsize = 18)
    cbr2.set_label(f'{component_nom} (mol/m²)', fontsize = 18)

    for i in range(2):
        
        ax[i].set_xlim([lon_min, lon_max])
        ax[i].set_ylim([lat_min, lat_max])
        ax[i].set_xlabel('Longitude', fontsize = 18)
        ax[i].set_ylabel('Latitude', fontsize = 18)
        ax[i].tick_params(labelsize = 16)

    ax[0].set_title('Original', fontsize = 20, pad = 20)
    ax[1].set_title('Interpolated', fontsize = 20, pad = 20)
    fig.suptitle('COMPONENT FOR CAMS AT HYBRID = 137 AT 12:00', fontsize = 22)
    plt.show()

    return CAMS_ds_new

In [ ]:
def TROPOMI_download(file_name, component_nom):

    # Access the API
    user = 's5pguest' 
    password = 's5pguest' 
    api = SentinelAPI(user, password, 'https://s5phub.copernicus.eu/dhus')

    # Get NetCDF file
    api.download(file_name, directory_path = 'data/tropomi/' + component_nom)

In [ ]:
def TROPOMI_read(product_name, component_nom):

    # Read as xarray dataset object
    TROPOMI_ds = xr.open_dataset('data/tropomi/' + component_nom + '/' + product_name, group = 'PRODUCT')

    # Change name of component
    TROPOMI_ds = TROPOMI_ds.rename({'nitrogendioxide_tropospheric_column': 'TROPOMI_column'})

    return TROPOMI_ds

In [ ]:
def visualize_pcolormesh(fig, axs, data_array, longitude, latitude, projection, color_scale, long_name, units, vmin, vmax, set_global=True, lonmin=-180, lonmax=180, latmin=-90, latmax=90):

    palette = copy(plt.get_cmap(color_scale))
    palette.set_under(alpha = 0)
    
    im = axs.pcolormesh(
                        longitude, latitude, data_array, 
                        cmap = palette, 
                        transform = projection,
                        vmin = vmin,
                        vmax = vmax,
                        norm = colors.Normalize(vmin = 0, vmax = vmax),
                        shading = 'auto'
                        )
                        
    axs.add_feature(cfeature.BORDERS, edgecolor = 'black', linewidth = 1)
    axs.add_feature(cfeature.COASTLINE, edgecolor = 'black', linewidth = 1)

    if (projection == ccrs.PlateCarree()):
        axs.set_extent([lonmin, lonmax, latmin, latmax], projection)
        gl = axs.gridlines(draw_labels = True, linestyle = '--')
        gl.top_labels = False
        gl.right_labels = False
        gl.xformatter = LONGITUDE_FORMATTER
        gl.yformatter = LATITUDE_FORMATTER
        gl.xlabel_style = {'size': 16}
        gl.ylabel_style = {'size': 16}

    if(set_global):
        axs.set_global()
        axs.gridlines()

    axs.set_title(long_name, fontsize = 18, pad = 20)
    axs.tick_params(labelsize = 14)

    cbr = fig.colorbar(im, ax = axs, extend = 'both', orientation = 'horizontal', fraction = 0.05, pad = 0.15)   
    cbr.set_label(units, fontsize = 16)
    cbr.ax.tick_params(labelsize = 14)
    cbr.ax.xaxis.get_offset_text().set_fontsize(14)